# Data Wranging -> Preparing Training Data

Our training data comes nicely to us from the spider database files 'train_spider.json' and 'train_others.json'.

The first was built by the team that prepared this data and the second is a collection based on common public datasets.

Included in the files are training questions with the corresponding schema_id, correct query to answer the question, and some parsed out data for the question and query.

I could actually pull the tokenized words from the question, but I want that practice so I'll do that. I also haven't thought of a good use-case to pull in the query information (I wouldn't know how to train on it) but I'll pull it anyway and then have a version of the data with full question, target schema, and target query. And then train off of only the 1st two.

In [2]:
import json
import pandas as pd

## Spider Training Data

### Load json

In [3]:
path = '../../src/data/raw/spider/'

with open(path+'train_spider.json', "r") as f:
    spi_train = json.load(f)

### Create Pandas DF
With question and corresponding schema

In [4]:
spi_train_list = []

for i in range(len(spi_train)):
    ques = spi_train[i]['question']
    schem = spi_train[i]['db_id']
    query = spi_train[i]['query']
    record = [ques, schem, query]
    spi_train_list.append(record)

In [5]:
spi_train_df = pd.DataFrame(spi_train_list, columns=['question', 'schema', 'query'])

In [6]:
spi_train_df

,question,schema,query
0,How many heads of the departments are older th...,department_management,SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...",department_management,"SELECT name , born_state , age FROM head ORD..."
2,"List the creation year, name and budget of eac...",department_management,"SELECT creation , name , budget_in_billions ..."
3,What are the maximum and minimum budget of the...,department_management,"SELECT max(budget_in_billions) , min(budget_i..."
4,What is the average number of employees of the...,department_management,SELECT avg(num_employees) FROM department WHER...
...,...,...,...
6995,What are all the company names that have a boo...,culture_company,SELECT T1.company_name FROM culture_company AS...
6996,Show the movie titles and book titles for all ...,culture_company,"SELECT T1.title , T3.book_title FROM movie AS..."
6997,What are the titles of movies and books corres...,culture_company,"SELECT T1.title , T3.book_title FROM movie AS..."
6998,Show all company names with a movie directed i...,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...


In [22]:
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("juierror/text-to-sql-with-table-schema")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/text-to-sql-with-table-schema")

def prepare_input(question: str, table: List[str]):
    table_prefix = "table:"
    question_prefix = "question:"
    join_table = ",".join(table)
    inputs = f"{question_prefix} {question} {table_prefix} {join_table}"
    input_ids = tokenizer(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, table: List[str]) -> str:
    input_data = prepare_input(question=question, table=table)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=700)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result
ques=input()
print(inference(question=ques, table=["id", "name", "age"]))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


SELECT COUNT name FROM table WHERE age = 15 to 22 AND name = city delhi


## Other Training Data

### Load json

In [7]:
path2 = '../../src/data/raw/spider/'

with open(path2+'train_others.json', "r") as f:
    oth_train = json.load(f)

### Create Pandas DF

With question and corresponding schema

In [8]:
oth_train_list = []

for i in range(len(oth_train)):
    ques = oth_train[i]['question']
    schem = oth_train[i]['db_id']
    query = oth_train[i]['query']
    record = [ques, schem, query]
    oth_train_list.append(record)

In [9]:
oth_train_df = pd.DataFrame(oth_train_list, columns=['question', 'schema','query'])

In [10]:
oth_train_df.head()

,question,schema,query
0,what is the biggest city in wyoming,geo,SELECT city_name FROM city WHERE population =...
1,what wyoming city has the largest population,geo,SELECT city_name FROM city WHERE population =...
2,what is the largest city in wyoming,geo,SELECT city_name FROM city WHERE population =...
3,where is the most populated area of wyoming,geo,SELECT city_name FROM city WHERE population =...
4,which city in wyoming has the largest population,geo,SELECT city_name FROM city WHERE population =...


## Combine Dataframes

In [11]:
full_train = pd.concat([spi_train_df, oth_train_df], axis=0, ignore_index=True)

In [12]:
#did the concat work. Let's check the counts
print(spi_train_df['question'].count())
print(oth_train_df['question'].count())
print(full_train['question'].count())

7000
1659
8659


That looks good! I'll now prep for the next step by saving this to the interim directory, and then create a df with just the first 2 columns for training and move it to the interim directory as well.

## Prep For Next Steps

### Save full data to the interim folder

In [13]:
#commenting out after running so I don't re-run unecessarily

#filepath = '../data/interim/full_training_data.csv'
#full_train.to_csv(filepath, index=False)

### Create Simpler Training File and Save to Interim Folder

In [14]:
#create new with just the first two columns
training_data = full_train[['question','schema']]

In [15]:
#look to make sure things copied correctly
print(training_data.head())
print('--'*50)
print(training_data.info())

                                            question                 schema
0  How many heads of the departments are older th...  department_management
1  List the name, born state and age of the heads...  department_management
2  List the creation year, name and budget of eac...  department_management
3  What are the maximum and minimum budget of the...  department_management
4  What is the average number of employees of the...  department_management
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8659 entries, 0 to 8658
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  8659 non-null   object
 1   schema    8659 non-null   object
dtypes: object(2)
memory usage: 135.4+ KB
None


In [16]:
#export csv
#filepath2 = '../data/interim/training_data.csv'
#training_data.to_csv(filepath2, index=False)